In [1]:
!pip install langchain-nvidia-ai-endpoints langchain langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.5.1 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2,

In [2]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
import datetime

In [3]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are an expert AI researcher.
            Currect time: {time}

            1. {first_instruction}
            2. Reflect and critique your answer. Be severe to maximize improvement.
            3. After the reflection, **list 1-3 search queries separately** for researching improvements.Do not include them inside the reflection.

            """,
        ),
        MessagesPlaceholder(variable_name = "messages"),
        ("system", "Answrt the user's question above using the required format.")
    ]
).partial(
    time = lambda: datetime.datetime.now().isoformat(),
)

In [4]:
!pip install pydantic

In [5]:
from pydantic import BaseModel, Field
from typing import List 

In [6]:
class Reflection(BaseModel):
    missing: str = Field(description = "Critique of what is missing.")
    superflous: str = Field(description = "Critque of what is superflous")

In [7]:
class AnswerQuestion(BaseModel):
    """ Answer the question """

    answer: str = Field(description = "~250 word detailed answer to the question.")
    search_queries: List[str] = Field(description = "1-3 search queries for answering improvements to address the critique of your current answer.")
    reflection: Reflection = Field(description = "Your reflection on the initial answer.")

In [8]:
first_responder_prompt_template = actor_prompt_template.partial(
    first_instruction = "Provide a detailed ~250 word answer"
)

In [9]:
from kaggle_secrets import UserSecretsClient

model = UserSecretsClient().get_secret("NVIDIA_MODEL_NAME")
api_key = UserSecretsClient().get_secret("NVIDIA_API_KEY")

In [10]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(
    model = model,
    api_key = api_key,
)

In [11]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

In [12]:
pydantic_parser = PydanticToolsParser(tools = [AnswerQuestion])

In [13]:
first_responder_chain = first_responder_prompt_template | llm.bind_tools(tools = [AnswerQuestion], tool_choice = 'AnswerQuestion') 

In [14]:
validator = PydanticToolsParser(tools = [AnswerQuestion])

In [15]:
from langchain_core.messages import HumanMessage

In [16]:
response = first_responder_chain.invoke({
    "messages": [HumanMessage(content = "Write me a blog post on how small business can leverage Ai to grow")]
})

In [17]:
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-c9f1d195f62b41d7b63d76bfa4ddaae2', 'type': 'function', 'function': {'name': 'AnswerQuestion', 'arguments': '{"answer": "Small businesses can leverage AI to grow in various ways, including automating administrative tasks, analyzing customer data to personalize marketing efforts, improving customer service through chatbots, and optimizing supply chains. AI can also help small businesses identify new opportunities and make data-driven decisions. Additionally, AI-powered tools can help small businesses streamline their operations and reduce costs. Overall, AI can be a powerful tool for small businesses looking to grow and compete in today\'s digital marketplace.", "search_queries": ["AI for small business", "small business growth strategies", "AI-powered tools for small business"], "reflection": {"missing": "More specific examples of AI-powered tools and platforms that small businesses can use would be helpful.", "superflou

In [18]:
print(first_responder_prompt_template)

input_variables=['messages'] input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.messages.function.Funct

## Revisor Section

In [19]:
revise_instruction = """ Revise your previous answer using the new information.
-- You should use the previous critique to add important information to your andswer.
    -- You must include numerical citations in your revised answer to ensure it can be verified.
    -- Add a "References" section to the bottom of your answer (which does not count towards the word linit). In form of:
        -- [1] https://example.com
        -- [2] https://example.com
    -- You should use the previous critique to remove superflous information from your answer and make sure it is not more than 250 words.
"""

In [20]:
class ReviseAnswer(AnswerQuestion):
    """ Revise your original answer tp your question """

    references: List[str] = Field(description = "Citations motivating your updated answer")

In [21]:
revisor_chain = actor_prompt_template.partial(
    first_instruction = revise_instruction
) | llm.bind_tools(tools = [ReviseAnswer], tool_choice = "ReviseAnswer")

## Execute Tools Section

In [22]:
import json
from typing import List, Dict, Any
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage
from langchain_community.tools import TavilySearchResults

In [23]:
tavily_api_key = UserSecretsClient().get_secret("TAVILY_API_KEY")
print(tavily_api_key[:9])

tvly-dev-


In [24]:
tavily_tool = TavilySearchResults(max_results = 5,
                                 tavily_api_key = tavily_api_key)

/tmp/ipykernel_13/2446320105.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results = 5,


In [25]:
def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    last_ai_message: AIMessage = state[-1]

    if not hasattr(last_ai_message, "tool_calls") or not last_ai_message.tool_calls:
        return []

    tool_messages = []

    for tool_call in last_ai_message.tool_calls:
        if tool_call["name"] in ["AnswerQuestion", "ReviseAnswer"]:
            call_id = tool_call["id"]
            search_queries = tool_call["args"].get("search_queries", [])

            query_results = {}
            for query in search_queries:
                result = tavily_tool.invoke(query)
                query_results[query] = result

            tool_messages.append(
                ToolMessage(
                    content = json.dumps(query_results),
                    tool_call_id = call_id
                )
            )

    return tool_messages

## Reflexion Graph

In [26]:
!pip install langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.2/153.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.66
    Uninstalling langchain-core-0.3.66:
      Successfully uninstalled langchain-core-0.3.66


In [27]:
from typing import List

from langchain_core.messages import BaseMessage, ToolMessage
from langgraph.graph import END, MessageGraph


In [28]:
graph = MessageGraph()

/tmp/ipykernel_13/1254895084.py:1: LangGraphDeprecatedSinceV10: MessageGraph is deprecated in LangGraph v1.0.0, to be removed in v2.0.0. Please use StateGraph with a `messages` key instead. Deprecated in LangGraph V1.0 to be removed in V2.0.
  graph = MessageGraph()


In [29]:
graph.add_node("draft", first_responder_chain)

In [30]:
graph.add_node("execute_tools", execute_tools)

In [31]:
graph.add_node("revisor", revisor_chain)

In [32]:
graph.add_edge("draft", "execute_tools")

In [33]:
graph.add_edge("execute_tools", "revisor")

In [34]:
MAX_ITERATIONS = 2

In [35]:
def event_loop(state: List[BaseMessage]) -> str:
    count_tool_visits = sum(isinstance(item, ToolMessage) for item in state)
    num_iterations = count_tool_visits
    if num_iterations > MAX_ITERATIONS:
        return END
    return "execute_tools"
        

In [36]:
graph.add_conditional_edges("revisor", event_loop)

In [37]:
graph.set_entry_point("draft")

In [38]:
app = graph.compile()

In [39]:
print(app.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	draft(draft)
	execute_tools(execute_tools)
	revisor(revisor)
	__end__([<p>__end__</p>]):::last
	__start__ --> draft;
	draft --> execute_tools;
	execute_tools --> revisor;
	revisor --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [40]:
response = app.invoke("Write about how small business can leverage AI to grow")

In [41]:
print(response[-1].tool_calls[0]["args"]["answer"])

AI can improve customer service by automating tasks and providing insights that support agents can use to make informed decisions. It can analyze customer data to identify trends, preferences, and common issues, enabling businesses to tailor their products and services to meet demand.", 


In [42]:
print(response)

[HumanMessage(content='Write about how small business can leverage AI to grow', additional_kwargs={}, response_metadata={}, id='8bfc1c0a-1011-4274-ab73-851b7a5d5d53'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-cc56f5c1fb95428692c07bee51c2a9c9', 'type': 'function', 'function': {'name': 'AnswerQuestion', 'arguments': '{"answer": "Small businesses can leverage AI to grow in various ways, including automation of manual tasks, data analysis, and customer service. AI can help automate tasks such as bookkeeping, inventory management, and marketing, freeing up time for business owners to focus on higher-level tasks. Additionally, AI-powered tools can analyze customer data to provide insights on behavior, preferences, and needs, enabling businesses to tailor their products and services to meet demand. AI-powered chatbots can also provide 24/7 customer support, improving customer satisfaction and loyalty. Furthermore, AI can help small businesses identify new 